##                                                                 迁移学习

迁移学习的好处就在于，将训练好的模型当成特征提取器，然后加上简单的神经网络层，用很小的训练代价就可以得到很好的结果；这个文档是机遇inception-v3和Mobilenet来展开的；图像数据集准备：http://download.tensorflow.org/example_images/flower_photos.tgz 模型准备：inceptionv3 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'; 若要运行的是Mobilenet, 会自动根据'http://download.tensorflow.org/models/mobilenet_v1_' + version_string + '_' + size_string + '_frozen.tgz'

当选择不变换图像时，会首先计算所有的bottlenecks，然后随机选取batch进行训练，这个过程还是比较快的；
若选择进行图像变换，那么会对每次进行训练的batch进行图像变换然后计算bottlenecks，最后再进行训练；
(1) 从原始模型得到的只有一个计算图和两个tensor：resized_image_tensor(输入)，bottleneck_tensor(输出)；
(2) 一幅图像到bottleneck的转化func8：image_string_data ->feed-> jpeg_data_tensor(输入) -> decoded_image_tensor(输入) ->feed-> resized_image_tensor(模型输入) -> bottleneck_tensor(结果输出) -> bottleneck;

子函数实现的功能：
1. 把样本中所有的图片列表并按训练、验证、测试数据分开
    create_image_lists(image_dir, testing_percentage, validation_percentage)；
2. 定义函数通过类别名称、所属数据集和图片编号获取一张图片的地址。
    get_image_path(image_lists, label_name, index, image_dir, category):
3. 定义函数生成模型处理之后的特征向量的文件地址。
    get_bottleneck_path(image_lists, label_name, index, bottleneck_dir, category, architecture):
4. 建立图模型。
    create_model_graph(model_info):
5. 定义函数使用加载的训练好的Inception-v3模型处理一张图片，得到这个图片的特征向量。
    run_bottleneck_on_image(sess, image_data, image_data_tensor,decoded_image_tensor, resized_input_tensor,  bottleneck_tensor):
6. 下载并解压文件。
    maybe_download_and_extract(data_url):
7. 确保一个folder存在。ensure_dir_exists(dir_name):
8. 生成一个bottleneck并保存:
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index, image_dir, category, sess, jpeg_data_tensor, decoded_image_tensor, resized_input_tensor, bottleneck_tensor):
9. 若bottleneck存在则进行读取，若不存在或者读取错误则重新生成，调用了8;
    get_or_create_bottleneck(sess, image_lists, label_name, index, image_dir, category, bottleneck_dir, jpeg_data_tensor, decoded_image_tensor, resized_input_tensor, bottleneck_tensor, architecture):
10. 对所有的图像生成bottleneck，并按照bottleneck_path写入文件中，调用了9;
   cache_bottlenecks(sess, image_lists, image_dir, bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
      resized_input_tensor, bottleneck_tensor, architecture): 
11. 得到某一category的how_many个随机bottlenecks或者全部bottlenecks，是对10得到的众多结果读取;
    get_random_cached_bottlenecks(sess, image_lists, how_many, category, bottleneck_dir, image_dir, jpeg_data_tensor,
       decoded_image_tensor, resized_input_tensor, bottleneck_tensor, architecture):
12. 对于某一种category，先将图像进行变换再重新生成bottleneck，得到how_many个bottlenecks和ground_truths;
    get_random_distorted_bottlenecks(sess, image_lists, how_many, category, image_dir, input_jpeg_tensor,
    distorted_image, resized_input_tensor, bottleneck_tensor):
13. 判断是否需要变换图像; should_distort_images(flip_left_right, random_crop, random_scale, random_brightness):
14. 设计图像变化流程，返回placeholder和变换结果tensor;
    add_input_distortions(flip_left_right, random_crop, random_scale,random_brightness, input_width, input_height,
      input_depth, input_mean, input_std):
15. 生成一个tensor的summary; variable_summaries(var):
16. 定义分类层，返回分类层的重要op; 
    add_final_training_ops(class_count, final_tensor_name, bottleneck_tensor, bottleneck_tensor_size):
17. 定义函数对比预测值与ground_truth，返回准确率和预测值; add_evaluation_step(result_tensor, ground_truth_tensor):
18. 将重新训练后的模型保存成二进制文件; save_graph_to_file(sess, graph, graph_file_name):
19. 为summary准备路径; prepare_file_system():
20. 输入模型名称，返回模型的信息; create_model_info(architecture):
21. 根据模型信息预处理图像; add_jpeg_decoding(input_width, input_height, input_depth, input_mean, input_std):

### 1. 把样本中所有的图片列表并按训练、验证、测试数据分开

和1.0 book中的函数功能是类似的，最终的目的是实现一个dictionary, 输入正整数的两个百分比，然后将每一类的图像进行划分
result = 
{
    "rose":{
              'dir': roses,
              'training': [1.jpg, 2.jpeg, 64.JPG, 83.JPEG, .....],
              'testing': [43.jpg, 93.jpeg, lala.JPG, haha.JPEG],
              'validation': [some image names....],
           }
    "sunflowers":{
              'dir':sunflowers,
              'training':[]
              'testing':[]
              'validation':[]
              }

    ......
}
得到上述的字典结构，其中result={"label1":{'dir':rose, }, ...}；label部分可能与子文件夹名称不同，但是dir部分是相同不变的；
不同之处：没有采用0~100随机数的方法来进行图像分类，而是采用的hashlib的方法，定义了最大hash值，然后进行的划分，结果是相同的
返回：result dict

In [ ]:
# These are all parameters that are tied to the particular model architecture
# we're using for Inception v3. These include things like tensor names and their
# sizes. If you want to adapt this script to work with another model, you will
# need to update these to reflect the values in the network you're using.
MAX_NUM_IMAGES_PER_CLASS = 2 ** 27 - 1  # ~134M


def create_image_lists(image_dir, testing_percentage, validation_percentage):
  """Builds a list of training images from the file system.
  Analyzes the sub folders in the image directory, splits them into stable
  training, testing, and validation sets, and returns a data structure
  describing the lists of images for each label and their paths.
  Args:
    image_dir: String path to a folder containing subfolders of images.
    testing_percentage: Integer percentage of the images to reserve for tests.
    validation_percentage: Integer percentage of images reserved for validation.
  Returns:
    A dictionary containing an entry for each label subfolder, with images split
    into training, testing, and validation sets within each label.
  """
  if not gfile.Exists(image_dir):
    tf.logging.error("Image directory '" + image_dir + "' not found.")
    return None
  result = {}
  sub_dirs = [x[0] for x in gfile.Walk(image_dir)]
  # The root directory comes first, so skip it.
  is_root_dir = True
  for sub_dir in sub_dirs:
    if is_root_dir:
      is_root_dir = False
      continue
    extensions = ['jpg', 'jpeg', 'JPG', 'JPEG']
    file_list = []
    dir_name = os.path.basename(sub_dir)
    if dir_name == image_dir:
      continue
    tf.logging.info("Looking for images in '" + dir_name + "'")
    for extension in extensions:
      file_glob = os.path.join(image_dir, dir_name, '*.' + extension)
      file_list.extend(gfile.Glob(file_glob))
    if not file_list:
      tf.logging.warning('No files found')
      continue
    if len(file_list) < 20:
      tf.logging.warning(
          'WARNING: Folder has less than 20 images, which may cause issues.')
    elif len(file_list) > MAX_NUM_IMAGES_PER_CLASS:
      tf.logging.warning(
          'WARNING: Folder {} has more than {} images. Some images will '
          'never be selected.'.format(dir_name, MAX_NUM_IMAGES_PER_CLASS))
    label_name = re.sub(r'[^a-z0-9]+', ' ', dir_name.lower())
    training_images = []
    testing_images = []
    validation_images = []
    for file_name in file_list:
      base_name = os.path.basename(file_name)
      # We want to ignore anything after '_nohash_' in the file name when
      # deciding which set to put an image in, the data set creator has a way of
      # grouping photos that are close variations of each other. For example
      # this is used in the plant disease data set to group multiple pictures of
      # the same leaf.
      hash_name = re.sub(r'_nohash_.*$', '', file_name)
      # This looks a bit magical, but we need to decide whether this file should
      # go into the training, testing, or validation sets, and we want to keep
      # existing files in the same set even if more files are subsequently
      # added.
      # To do that, we need a stable way of deciding based on just the file name
      # itself, so we do a hash of that and then use that to generate a
      # probability value that we use to assign it.
      hash_name_hashed = hashlib.sha1(compat.as_bytes(hash_name)).hexdigest()
      percentage_hash = ((int(hash_name_hashed, 16) %
                          (MAX_NUM_IMAGES_PER_CLASS + 1)) *
                         (100.0 / MAX_NUM_IMAGES_PER_CLASS))
      if percentage_hash < validation_percentage:
        validation_images.append(base_name)
      elif percentage_hash < (testing_percentage + validation_percentage):
        testing_images.append(base_name)
      else:
        training_images.append(base_name)
    result[label_name] = {
        'dir': dir_name,
        'training': training_images,
        'testing': testing_images,
        'validation': validation_images,
    }
  return result



### 2. 定义函数通过类别名称、所属数据集和图片编号获取一张图片的地址。


#def get_image_path(image_lists, image_dir, label_name, index, category):
def get_image_path(image_lists, label_name, index, image_dir, category):
    这个函数的作用就是根据上述的result，也就是参数中的image_lists，图像的存储路径image_dir:/path/flower_photos；标签名称label_name: roses, sunflowers等； category: training, or testing,or validation; index: 具体的索引下标; 来唯一的获取一张图片
也就是将image_lists中存储的路径，结合image_dir的路径，并根据index来唯一索引到一张图片；
    image_list -> result
    label_name -> roeses, or sunflowers...
    index -> number
    image_dir -> /path/flower_photos
    category -> training, or testing, or validation
返回：一张图像的完整路径；
上面是book中的函数参数，这个函数的功能是一致的，根据输入的label_name, 整数的index，image_dir, 训练模式category返回一幅图像的绝对路径；

In [ ]:
def get_image_path(image_lists, label_name, index, image_dir, category):
  """"Returns a path to an image for a label at the given index.
  Args:
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Int offset of the image we want. This will be moduloed by the
    available number of images for the label, so it can be arbitrarily large.
    image_dir: Root folder string of the subfolders containing the training
    images.
    category: Name string of set to pull images from - training, testing, or
    validation.
  Returns:
    File system path string to an image that meets the requested parameters.
  """
  if label_name not in image_lists:
    tf.logging.fatal('Label does not exist %s.', label_name)
  label_lists = image_lists[label_name]
  if category not in label_lists:
    tf.logging.fatal('Category does not exist %s.', category)
  category_list = label_lists[category]
  if not category_list:
    tf.logging.fatal('Label %s has no images in the category %s.',
                     label_name, category)
  mod_index = index % len(category_list)
  base_name = category_list[mod_index]
  sub_dir = label_lists['dir']
  full_path = os.path.join(image_dir, sub_dir, base_name)
  return full_path

### 3. 定义函数生成模型处理之后的特征向量的文件地址。

def get_bottleneck_path(image_lists, label_name, index, bottleneck_dir, category, architecture):
  return get_image_path(image_lists, label_name, index, bottleneck_dir, category) + '_' + architecture + '.txt'
    将模型的输出当成了特征提取过程，每个图像都会得到一个特征向量，保存成与图像名对应的txt文件：image-name.jpg_modelname.txt
    image_lists: 1中得到的图像的名称，位置，分类等信息的字典；
    label_name: 每一类的名称roses, sunflowers....
    index: 每一类下的某一张图像索引；
    bottleneck_dir: 保存特征向量的根路径；
    category: training, testing, validation
    architecture：由于可以训练inception-v3或者别的不同Mobilenet, 这里加了这个作为区分；
    首先根据image_list, label_name, inxex, category来得到: label_name/image-name.jpg;
    然后根据bottleneck_dir生成：bottleneck_dir/labe_name/image-name.jpg_inception_v3.txt
    返回：一个txt文件的路径名称；

In [ ]:
def get_bottleneck_path(image_lists, label_name, index, bottleneck_dir,
                        category, architecture):
  """"Returns a path to a bottleneck file for a label at the given index.
  Args:
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Integer offset of the image we want. This will be moduloed by the
    available number of images for the label, so it can be arbitrarily large.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    category: Name string of set to pull images from - training, testing, or
    validation.
    architecture: The name of the model architecture.
  Returns:
    File system path string to an image that meets the requested parameters.
  """
  return get_image_path(image_lists, label_name, index, bottleneck_dir,
                        category) + '_' + architecture + '.txt'

### 4. 建立图模型

根据输入的model_info生成一个graph，并返回graph和两个tensor；
输入：model_info 保存了图像的结构信息；
返回：包含已训练模型的网络，和两个tensor：bottleneck_tensor, resized_input_tensor

In [ ]:
def create_model_graph(model_info):
  """"Creates a graph from saved GraphDef file and returns a Graph object.
  Args:
    model_info: Dictionary containing information about the model architecture.
  Returns:
    Graph holding the trained Inception network, and various tensors we'll be
    manipulating.
  """
  with tf.Graph().as_default() as graph:
    model_path = os.path.join(FLAGS.model_dir, model_info['model_file_name'])
    with gfile.FastGFile(model_path, 'rb') as f:
      graph_def = tf.GraphDef()
      graph_def.ParseFromString(f.read())
      bottleneck_tensor, resized_input_tensor = (tf.import_graph_def(
          graph_def,
          name='',
          return_elements=[
              model_info['bottleneck_tensor_name'],
              model_info['resized_input_tensor_name'],
          ]))
  return graph, bottleneck_tensor, resized_input_tensor

### 5. 定义函数使用加载的训练好的Inception-v3模型处理一张图片，得到这个图片的特征向量。

用模型的bottleneck层对输入的图像计算特征，返回计算好的特征向量；
image_data: 输入图像,gfile读取image得到的string文件；
image_data_tensor: 定义好的placeholder，图中的的数据输入层；
decoded_image_tensor: 输出改变尺寸和预处理之后的op；
resized_input_tensor: placeholder, 接受上述decoded之后的图像tensor
bottleneck_tensor: 原图中的bottleneck层，softmax之前的层；
返回：计算得到的特征向量bottleneck_values;
image_string_data ->feed-> jpeg_data_tensor(image_data_tensor) -> decoded_image_tensor ->feed-> resized_image_tensor 
-> bottleneck_tensor -> bottleneck;

In [ ]:
def run_bottleneck_on_image(sess, image_data, image_data_tensor,
                            decoded_image_tensor, resized_input_tensor,
                            bottleneck_tensor):
  """Runs inference on an image to extract the 'bottleneck' summary layer.
  Args:
    sess: Current active TensorFlow Session.
    image_data: String of raw JPEG data.
    image_data_tensor: Input data layer in the graph.
    decoded_image_tensor: Output of initial image resizing and  preprocessing.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: Layer before the final softmax.
  Returns:
    Numpy array of bottleneck values.
  """
  # First decode the JPEG image, resize it, and rescale the pixel values.
  resized_input_values = sess.run(decoded_image_tensor,
                                  {image_data_tensor: image_data})
  # Then run it through the recognition network.
  bottleneck_values = sess.run(bottleneck_tensor,
                               {resized_input_tensor: resized_input_values})
  bottleneck_values = np.squeeze(bottleneck_values)
  return bottleneck_values

### 6. 下载并解压文件

首先判断需要的训练好的模型是否在路径中，若没有则从url上download；
输入：data_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'; 是保存在model info中的，
同时，若要运行的是Mobilenet, 会自动根据data_url = 'http://download.tensorflow.org/models/mobilenet_v1_' + version_string + '_' + size_string + '_frozen.tgz'
来自动生成Mobile的下载路径；
默认模型保存文件夹：/tmp/imagenet；
默认模型路径：/tmp/imagenet/inception-2015-12-05.tgz 或跟上Mobilenet；
若默认路径不存在则进行下载

In [ ]:
def maybe_download_and_extract(data_url):
  """Download and extract model tar file.
  If the pretrained model we're using doesn't already exist, this function
  downloads it from the TensorFlow.org website and unpacks it into a directory.
  Args:
    data_url: Web location of the tar file containing the pretrained model.
  """
  dest_directory = FLAGS.model_dir #/tmp/imagenet, change with --model_dir some_dir
  if not os.path.exists(dest_directory):
    os.makedirs(dest_directory)
  filename = data_url.split('/')[-1]
  filepath = os.path.join(dest_directory, filename)
  if not os.path.exists(filepath):

    def _progress(count, block_size, total_size):
      sys.stdout.write('\r>> Downloading %s %.1f%%' %
                       (filename,
                        float(count * block_size) / float(total_size) * 100.0))
      sys.stdout.flush()

    filepath, _ = urllib.request.urlretrieve(data_url, filepath, _progress)
    print()
    statinfo = os.stat(filepath)
    tf.logging.info('Successfully downloaded', filename, statinfo.st_size,
                    'bytes.')
  tarfile.open(filepath, 'r:gz').extractall(dest_directory)

### 7. 确保一个folder存在

输入一个direcotry名称，若不存在则mkdir

In [ ]:
def ensure_dir_exists(dir_name):
  """Makes sure the folder exists on disk.
  Args:
    dir_name: Path string to the folder we want to create.
  """
  if not os.path.exists(dir_name):
    os.makedirs(dir_name)

### 8. 生成一个bottleneck并保存

bottleneck_path: 要保存的bottleneck的txt文件路径，可通过get_bottleneck_path获取；
image_list: 图像dictionary；
label_name: 用来在image_list中索引某一类；
category: 用来在某一类中确定是training, testing, validation等；
index: 用来在training, testing, validation数据集中确定是某一张图像；
image_dir: /path/to/flower_pthoto;
sess:
jpeg_data_tensor: 也即image_data_tensor, placeholder for image gfile string;
decoded_image_tensor: decode, resize, rescale op;
resized_input_tensor: placeholder for the output of decoded_image_tensor；
bottleneck_tensor: 计算得到的特征向量；
注意的是，这里并没有返回值，直接写入了文件，所以这个版本的操作是分成两部分进行的，先生成所有的文件，然后再进行训练；

image_string_data ->feed-> jpeg_data_tensor -> decoded_image_tensor ->feed-> resized_image_tensor -> bottleneck_tensor -> bottleneck;

In [ ]:
def create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor):
  """Create a single bottleneck file."""
  tf.logging.info('Creating bottleneck at ' + bottleneck_path)
  image_path = get_image_path(image_lists, label_name, index,
                              image_dir, category)
  if not gfile.Exists(image_path):
    tf.logging.fatal('File does not exist %s', image_path)
  image_data = gfile.FastGFile(image_path, 'rb').read()
  try:
    bottleneck_values = run_bottleneck_on_image(
        sess, image_data, jpeg_data_tensor, decoded_image_tensor,
        resized_input_tensor, bottleneck_tensor)
  except Exception as e:
    raise RuntimeError('Error during processing file %s (%s)' % (image_path,
                                                                 str(e)))
  bottleneck_string = ','.join(str(x) for x in bottleneck_values)
  with open(bottleneck_path, 'w') as bottleneck_file:
    bottleneck_file.write(bottleneck_string)

### 9. 若bottleneck存在则进行读取，若不存在或者读取错误则重新生成

函数调用get_bottleneck_path获取bottleneck文件路径，当文件不存在或者读取错误时重新生成文件create_bottleneck_file; 注意的是这个函数是得到一个bottleneck 特征向量；
一个原始图像到bottleneck的流程是：
image_string_data ->feed-> jpeg_data_tensor -> decoded_image_tensor ->feed-> resized_image_tensor -> bottleneck_tensor -> bottleneck;
其中，在jpeg_data_tensor计算得到decoded_image_tensor的时候会将图像resize成模型需要的resized_image_tensor尺寸大小；

In [ ]:
def get_or_create_bottleneck(sess, image_lists, label_name, index, image_dir,
                             category, bottleneck_dir, jpeg_data_tensor,
                             decoded_image_tensor, resized_input_tensor,
                             bottleneck_tensor, architecture):
  """Retrieves or calculates bottleneck values for an image.
  If a cached version of the bottleneck data exists on-disk, return that,
  otherwise calculate the data and save it to disk for future use.
  Args:
    sess: The current active TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    label_name: Label string we want to get an image for.
    index: Integer offset of the image we want. This will be modulo-ed by the
    available number of images for the label, so it can be arbitrarily large.
    image_dir: Root folder string  of the subfolders containing the training
    images.
    category: Name string of which  set to pull images from - training, testing,
    or validation.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    jpeg_data_tensor: The tensor to feed loaded jpeg data into.
    decoded_image_tensor: The output of decoding and resizing the image.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The output tensor for the bottleneck values.
    architecture: The name of the model architecture.
  Returns:
    Numpy array of values produced by the bottleneck layer for the image.
  """
  label_lists = image_lists[label_name]
  sub_dir = label_lists['dir']
  sub_dir_path = os.path.join(bottleneck_dir, sub_dir)
  ensure_dir_exists(sub_dir_path)
  bottleneck_path = get_bottleneck_path(image_lists, label_name, index,
                                        bottleneck_dir, category, architecture)
  if not os.path.exists(bottleneck_path):
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor)
  with open(bottleneck_path, 'r') as bottleneck_file:
    bottleneck_string = bottleneck_file.read()
  did_hit_error = False
  try:
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  except ValueError:
    tf.logging.warning('Invalid float found, recreating bottleneck')
    did_hit_error = True
  if did_hit_error:
    create_bottleneck_file(bottleneck_path, image_lists, label_name, index,
                           image_dir, category, sess, jpeg_data_tensor,
                           decoded_image_tensor, resized_input_tensor,
                           bottleneck_tensor)
    with open(bottleneck_path, 'r') as bottleneck_file:
      bottleneck_string = bottleneck_file.read()
    # Allow exceptions to propagate here, since they shouldn't happen after a
    # fresh creation
    bottleneck_values = [float(x) for x in bottleneck_string.split(',')]
  return bottleneck_values

### 10. 对所有的图像生成bottleneck，并按照bottleneck_path写入文件中

计算bottleneck特征向量是十分耗时的，训练的过程中需要多次用到所有的特征向量，每次都重新生成显然是不合适的；可选的做法是，将所有的特征向量一次生成，然后在训练的过程中多次读取；这个函数的功能就是遍历所有的图像，计算得到特征向量，并保存。
image_list: dictionary for all image.
image_dir: /path/to/flower_photos；
bottleneck_dir: 保存特征向量的根路径；
其余参数看代码解释；
执行过程：根据image_list得到文件路径结构，对于每一类，每一类中的training, testing, validation都生成feature vector保存；得到如下结构：
/tmp/bottleneck/：daisy  dandelion  roses  sunflowers  tulips；
这里有疑问的是，只传入了一个bottleneck_dir，就计算了所有的特征向量，是如何分成不同的字文件夹的？
答案：在get_or_create_bottleneck中生成之前会判断sub_dir是否存在；
一个原始图像到bottleneck的流程是：
image_string_data ->feed-> jpeg_data_tensor -> decoded_image_tensor ->feed-> resized_image_tensor -> bottleneck_tensor -> bottleneck

In [ ]:
def cache_bottlenecks(sess, image_lists, image_dir, bottleneck_dir,
                      jpeg_data_tensor, decoded_image_tensor,
                      resized_input_tensor, bottleneck_tensor, architecture):
  """Ensures all the training, testing, and validation bottlenecks are cached.
  Because we're likely to read the same image multiple times (if there are no
  distortions applied during training) it can speed things up a lot if we
  calculate the bottleneck layer values once for each image during
  preprocessing, and then just read those cached values repeatedly during
  training. Here we go through all the images we've found, calculate those
  values, and save them off.
  Args:
    sess: The current active TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    image_dir: Root folder string of the subfolders containing the training
    images.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    jpeg_data_tensor: Input tensor for jpeg data from file.
    decoded_image_tensor: The output of decoding and resizing the image.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The penultimate output layer of the graph.
    architecture: The name of the model architecture.
  Returns:
    Nothing.
这里有疑问的是，只传入了一个bottleneck_dir，就计算了所有的特征向量，是如何分成不同的字文件夹的？
  """
  how_many_bottlenecks = 0
  ensure_dir_exists(bottleneck_dir) # if not, mkdir
  for label_name, label_lists in image_lists.items(): #label_name: different classes, label_lists: contents
    for category in ['training', 'testing', 'validation']:
      category_list = label_lists[category]
      for index, unused_base_name in enumerate(category_list):
        # get_or_create_bottleneck 函数中ensure_dir_exists(sub_dir_path)保证了每个子folder的存在
        get_or_create_bottleneck(
            sess, image_lists, label_name, index, image_dir, category,
            bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
            resized_input_tensor, bottleneck_tensor, architecture)

        how_many_bottlenecks += 1
        if how_many_bottlenecks % 100 == 0:
          tf.logging.info(
              str(how_many_bottlenecks) + ' bottleneck files created.')

### 11. 得到某一category的how_many个随机bottlenecks或者全部bottlenecks

这个函数的功能主要是从cache_bottlenecks中生成好的bottlenecks中提取出某一个category的一部分或者全部；
这里需要注意的是，参数中指定了how_many和category；category指定了读取training, testing还是validation；how_many>=0需要读取how_many个，至于是某个label_name类别中的category，以及是某个category中的某一幅图像图像都是随机的；当how_many< 0时，全部读取所有label_name中的指定category；这么做的好处就是training的时候当作batch用，testing的时候全部读取；
返回：读取的bottlenecks，与之对应的ground_truths，与之对应的图像名称filenames；

In [ ]:
def get_random_cached_bottlenecks(sess, image_lists, how_many, category,
                                  bottleneck_dir, image_dir, jpeg_data_tensor,
                                  decoded_image_tensor, resized_input_tensor,
                                  bottleneck_tensor, architecture):
  """Retrieves bottleneck values for cached images.
  If no distortions are being applied, this function can retrieve the cached
  bottleneck values directly from disk for images. It picks a random set of
  images from the specified category.
  Args:
    sess: Current TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    how_many: If positive, a random sample of this size will be chosen.
    If negative, all bottlenecks will be retrieved.
    category: Name string of which set to pull from - training, testing, or
    validation.
    bottleneck_dir: Folder string holding cached files of bottleneck values.
    image_dir: Root folder string of the subfolders containing the training
    images.
    下面的tensor在需要生成bottleneck时用得到
    jpeg_data_tensor: The layer to feed jpeg image data into.
    decoded_image_tensor: The output of decoding and resizing the image.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The bottleneck output layer of the CNN graph.
    architecture: The name of the model architecture.
  Returns:
    List of bottleneck arrays, their corresponding ground truths, and the
    relevant filenames.
  """
  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  filenames = []
  if how_many >= 0:
    # Retrieve a random sample of bottlenecks.
    for unused_i in range(how_many):
      label_index = random.randrange(class_count)
      label_name = list(image_lists.keys())[label_index] #random label class
      image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
      image_name = get_image_path(image_lists, label_name, image_index,
                                  image_dir, category) #random image_name in this category
      bottleneck = get_or_create_bottleneck(
          sess, image_lists, label_name, image_index, image_dir, category,
          bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
          resized_input_tensor, bottleneck_tensor, architecture)
      ground_truth = np.zeros(class_count, dtype=np.float32)
      ground_truth[label_index] = 1.0
      bottlenecks.append(bottleneck)
      ground_truths.append(ground_truth)
      filenames.append(image_name)
  else:
    # Retrieve all bottlenecks.
    for label_index, label_name in enumerate(image_lists.keys()):
      for image_index, image_name in enumerate(
          image_lists[label_name][category]):
        image_name = get_image_path(image_lists, label_name, image_index,
                                    image_dir, category)
        bottleneck = get_or_create_bottleneck(
            sess, image_lists, label_name, image_index, image_dir, category,
            bottleneck_dir, jpeg_data_tensor, decoded_image_tensor,
            resized_input_tensor, bottleneck_tensor, architecture)
        ground_truth = np.zeros(class_count, dtype=np.float32)
        ground_truth[label_index] = 1.0
        bottlenecks.append(bottleneck)
        ground_truths.append(ground_truth)
        filenames.append(image_name)
  return bottlenecks, ground_truths, filenames

### 12. 对于某一种category，先将图像进行变换再重新生成bottleneck，得到how_many个bottlenecks和ground_truths

当我们用变换后的图像(裁减，缩放，翻转)做训练时，我们需要对每幅图像重新计算，不能应用计算好的cached bottleneck；
这里多出了distored_image，首先input_jpeg_tensor作为input placeholder传入读取的图像；其次，用distored_image op对图像进行变换；再次，对变换后的图像进行bottleneck生成；最后组合成bottlenecks和ground_truths返回；整体的过程相当于，对于某一种category，随机选取某一个label_name中的一幅图像，进行图像变换，生成bottleneck和ground_truth;

In [ ]:
def get_random_distorted_bottlenecks(
    sess, image_lists, how_many, category, image_dir, input_jpeg_tensor,
    distorted_image, resized_input_tensor, bottleneck_tensor):
  """Retrieves bottleneck values for training images, after distortions.
  If we're training with distortions like crops, scales, or flips, we have to
  recalculate the full model for every image, and so we can't use cached
  bottleneck values. Instead we find random images for the requested category,
  run them through the distortion graph, and then the full graph to get the
  bottleneck results for each.
  Args:
    sess: Current TensorFlow Session.
    image_lists: Dictionary of training images for each label.
    how_many: The integer number of bottleneck values to return.
    category: Name string of which set of images to fetch - training, testing,
    or validation.
    image_dir: Root folder string of the subfolders containing the training
    images.
    input_jpeg_tensor: The input layer we feed the image data to.
    distorted_image: The output node of the distortion graph.
    resized_input_tensor: The input node of the recognition graph.
    bottleneck_tensor: The bottleneck output layer of the CNN graph.
  Returns:
    List of bottleneck arrays and their corresponding ground truths.
  """
  class_count = len(image_lists.keys())
  bottlenecks = []
  ground_truths = []
  for unused_i in range(how_many):
    label_index = random.randrange(class_count)
    label_name = list(image_lists.keys())[label_index]
    image_index = random.randrange(MAX_NUM_IMAGES_PER_CLASS + 1)
    image_path = get_image_path(image_lists, label_name, image_index, image_dir,
                                category)
    if not gfile.Exists(image_path):
      tf.logging.fatal('File does not exist %s', image_path)
    jpeg_data = gfile.FastGFile(image_path, 'rb').read()
    # Note that we materialize the distorted_image_data as a numpy array before
    # sending running inference on the image. This involves 2 memory copies and
    # might be optimized in other implementations.
    distorted_image_data = sess.run(distorted_image,
                                    {input_jpeg_tensor: jpeg_data})
    bottleneck_values = sess.run(bottleneck_tensor,
                                 {resized_input_tensor: distorted_image_data})
    bottleneck_values = np.squeeze(bottleneck_values)
    ground_truth = np.zeros(class_count, dtype=np.float32)
    ground_truth[label_index] = 1.0
    bottlenecks.append(bottleneck_values)
    ground_truths.append(ground_truth)
  return bottlenecks, ground_truths

### 13. 判断是否需要变换图像

若左右翻转为true，或者crop box中的边界百分比不为0，或者缩放的百分比不为0，或者亮度变化的百分比不为0，则需要变换，否则不需要；

In [ ]:
def should_distort_images(flip_left_right, random_crop, random_scale,
                          random_brightness):
  """Whether any distortions are enabled, from the input flags.
  Args:
    flip_left_right: Boolean whether to randomly mirror images horizontally.
    random_crop: Integer percentage setting the total margin used around the
    crop box.
    random_scale: Integer percentage of how much to vary the scale by.
    random_brightness: Integer range to randomly multiply the pixel values by.
  Returns:
    Boolean value indicating whether any distortions should be applied.
  """
  return (flip_left_right or (random_crop != 0) or (random_scale != 0) or
          (random_brightness != 0))

### 14. 设计图像变化流程，返回placeholder和变换结果tensor

对图像进行随机变换，包括左右翻转，裁减，缩放等；返回一个接收图像string的placeholder和一个变换后的结果tensor。
这里的input_width, input_height, input_depth, input_mean, input_std都来自于模型model_info，在做完图像变换后返回的图像tensor是模型计算bottleneck所需要的尺寸(类似于21 add_jpeg_decoding函数所做的部分)，inception_v3是[229,229,3]；

In [ ]:
def add_input_distortions(flip_left_right, random_crop, random_scale,
                          random_brightness, input_width, input_height,
                          input_depth, input_mean, input_std):
  """Creates the operations to apply the specified distortions.
  During training it can help to improve the results if we run the images
  through simple distortions like crops, scales, and flips. These reflect the
  kind of variations we expect in the real world, and so can help train the
  model to cope with natural data more effectively. Here we take the supplied
  parameters and construct a network of operations to apply them to an image.
  Cropping
  ~~~~~~~~
  Cropping is done by placing a bounding box at a random position in the full
  image. The cropping parameter controls the size of that box relative to the
  input image. If it's zero, then the box is the same size as the input and no
  cropping is performed. If the value is 50%, then the crop box will be half the
  width and height of the input. In a diagram it looks like this:
  <       width         >
  +---------------------+
  |                     |
  |   width - crop%     |
  |    <      >         |
  |    +------+         |
  |    |      |         |
  |    |      |         |
  |    |      |         |
  |    +------+         |
  |                     |
  |                     |
  +---------------------+
  Scaling
  ~~~~~~~
  Scaling is a lot like cropping, except that the bounding box is always
  centered and its size varies randomly within the given range. For example if
  the scale percentage is zero, then the bounding box is the same size as the
  input and no scaling is applied. If it's 50%, then the bounding box will be in
  a random range between half the width and height and full size.
  Args:
    flip_left_right: Boolean whether to randomly mirror images horizontally.
    random_crop: Integer percentage setting the total margin used around the
    crop box.
    random_scale: Integer percentage of how much to vary the scale by.
    random_brightness: Integer range to randomly multiply the pixel values by.
    graph.
    input_width: Horizontal size of expected input image to model.
    input_height: Vertical size of expected input image to model.
    input_depth: How many channels the expected input image should have.
    input_mean: Pixel value that should be zero in the image for the graph.
    input_std: How much to divide the pixel values by before recognition.
  Returns:
    The jpeg input layer and the distorted result tensor.
  """

  jpeg_data = tf.placeholder(tf.string, name='DistortJPGInput')#placeholde for tf.gfile.FastGFile read string
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth) #
  decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0) #变成4维
  margin_scale = 1.0 + (random_crop / 100.0)
  resize_scale = 1.0 + (random_scale / 100.0)
  margin_scale_value = tf.constant(margin_scale)
  resize_scale_value = tf.random_uniform(tensor_shape.scalar(),
                                         minval=1.0,
                                         maxval=resize_scale)
  scale_value = tf.multiply(margin_scale_value, resize_scale_value)
  precrop_width = tf.multiply(scale_value, input_width)
  precrop_height = tf.multiply(scale_value, input_height)
  precrop_shape = tf.stack([precrop_height, precrop_width])
  precrop_shape_as_int = tf.cast(precrop_shape, dtype=tf.int32)
  precropped_image = tf.image.resize_bilinear(decoded_image_4d,
                                              precrop_shape_as_int)
  precropped_image_3d = tf.squeeze(precropped_image, squeeze_dims=[0])
  cropped_image = tf.random_crop(precropped_image_3d,
                                 [input_height, input_width, input_depth])
  if flip_left_right:
    flipped_image = tf.image.random_flip_left_right(cropped_image)
  else:
    flipped_image = cropped_image
  brightness_min = 1.0 - (random_brightness / 100.0)
  brightness_max = 1.0 + (random_brightness / 100.0)
  brightness_value = tf.random_uniform(tensor_shape.scalar(),
                                       minval=brightness_min,
                                       maxval=brightness_max)
  brightened_image = tf.multiply(flipped_image, brightness_value)
  offset_image = tf.subtract(brightened_image, input_mean)
  mul_image = tf.multiply(offset_image, 1.0 / input_std)
  distort_result = tf.expand_dims(mul_image, 0, name='DistortResult')
  return jpeg_data, distort_result

### 15. 生成一个tensor的summary

输入一个tensor，得到其4个summary

In [ ]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

### 16. 定义分类层，返回分类层的重要op

在inception-v3(或者Mobilenet)模型的bottleneck层后面加上一个全连接分类层。所以需要输入bottleneck_tensor，以及其大小：bottleneck_tensor_size；final_tensor_name是添加的分类层的softmax层的名称；class_count用来定义ground_truth_input这个placeholde的size。中间也定义了许多summary
返回：
(1)分类层的train_Step(optimizer)；
(2)损失函数：cross_entropy_mean；
(3)由bottleneck_tensor得到的bottleneck_input [None, bottleneck_tensor_size]; 
(4)定义的ground_truth_input [None, class_count]；
(5)以及final_tensor, softmax output
 #根据bottleneck_tensor返回新加层的运算op和placeholder
 #bottleneck_input 是输入placeholder[None, bottleneck_tensor_size(2048 or 1001)]
 #ground_truth_input 是输入placeholder[None, len(image_lists.keys())=5]
 #cross_entropy 是损失函数，需要上述两个placeholder输入的数据；
 #train_step 是cross_entropy的优化op；

In [ ]:
def add_final_training_ops(class_count, final_tensor_name, bottleneck_tensor,
                           bottleneck_tensor_size):
  """Adds a new softmax and fully-connected layer for training.
  We need to retrain the top layer to identify our new classes, so this function
  adds the right operations to the graph, along with some variables to hold the
  weights, and then sets up all the gradients for the backward pass.
  The set up for the softmax and fully-connected layers is based on:
  https://www.tensorflow.org/versions/master/tutorials/mnist/beginners/index.html
  Args:
    class_count: Integer of how many categories of things we're trying to
    recognize.
    final_tensor_name: Name string for the new final node that produces results.
    bottleneck_tensor: The output of the main CNN graph.
    bottleneck_tensor_size: How many entries in the bottleneck vector.
  Returns:
    The tensors for the training and cross entropy results, and tensors for the
    bottleneck input and ground truth input.
  """
  with tf.name_scope('input'):
    bottleneck_input = tf.placeholder_with_default(
        bottleneck_tensor,
        shape=[None, bottleneck_tensor_size],
        name='BottleneckInputPlaceholder')

    ground_truth_input = tf.placeholder(tf.float32,
                                        [None, class_count],
                                        name='GroundTruthInput')

  # Organizing the following ops as `final_training_ops` so they're easier
  # to see in TensorBoard
  layer_name = 'final_training_ops'
  with tf.name_scope(layer_name):
    with tf.name_scope('weights'):
      initial_value = tf.truncated_normal(
          [bottleneck_tensor_size, class_count], stddev=0.001)

      layer_weights = tf.Variable(initial_value, name='final_weights')

      variable_summaries(layer_weights)
    with tf.name_scope('biases'):
      layer_biases = tf.Variable(tf.zeros([class_count]), name='final_biases')
      variable_summaries(layer_biases)
    with tf.name_scope('Wx_plus_b'):
      logits = tf.matmul(bottleneck_input, layer_weights) + layer_biases
      tf.summary.histogram('pre_activations', logits)

  final_tensor = tf.nn.softmax(logits, name=final_tensor_name)
  tf.summary.histogram('activations', final_tensor)

  with tf.name_scope('cross_entropy'):
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(
        labels=ground_truth_input, logits=logits)
    with tf.name_scope('total'):
      cross_entropy_mean = tf.reduce_mean(cross_entropy)
  tf.summary.scalar('cross_entropy', cross_entropy_mean)

  with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(FLAGS.learning_rate)
    train_step = optimizer.minimize(cross_entropy_mean)

  return (train_step, cross_entropy_mean, bottleneck_input, ground_truth_input,
          final_tensor)



### 17. 定义函数对比预测值与ground_truth，返回准确率和预测值

这个函数是作为测试和验证存在的；result_tensor是add_final_training_ops返回的final_tensor，也就是softmax(logits)；返回prediction是分类层的预测值argmax, evaluation_step是得到准确率的tensor

In [ ]:
def add_evaluation_step(result_tensor, ground_truth_tensor):
  """Inserts the operations we need to evaluate the accuracy of our results.
  Args:
    result_tensor: The new final node that produces results.
    ground_truth_tensor: The node we feed ground truth data
    into.
  Returns:
    Tuple of (evaluation step, prediction).
  """
  with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
      prediction = tf.argmax(result_tensor, 1)
      correct_prediction = tf.equal(
          prediction, tf.argmax(ground_truth_tensor, 1))
    with tf.name_scope('accuracy'):
      evaluation_step = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
  tf.summary.scalar('accuracy', evaluation_step)
  return evaluation_step, prediction

### 18. 将重新训练后的模型保存成二进制文件

将模型保存成二进制文件，首先转化成了constant; 然后将inception和添加的分类层都保存近了模型，84M; 注意这里有一个隐含参数，FLAGS.final_tensor_name，默认是 “final_result”， 目前还不清楚这个final tensor是做什么用的; 看代码发现这个FLAGS.final_tensor_name被add_final_training_ops应用到final_tensor=softmax(logits)，也就是最后的分类预测层，考虑是保存成二进制后又留下接口，这样的话是不是可以在保存的时候多留几个接口，以上只是我的猜测，附上api：https://www.tensorflow.org/api_docs/python/tf/graph_util/convert_variables_to_constants
参数：sess 当前的sess； graph 当前的计算图；graph_file_name 保存成的图像名称；

In [ ]:
def save_graph_to_file(sess, graph, graph_file_name):
  output_graph_def = graph_util.convert_variables_to_constants(
      sess, graph.as_graph_def(), [FLAGS.final_tensor_name])
  with gfile.FastGFile(graph_file_name, 'wb') as f:
    f.write(output_graph_def.SerializeToString())
  return

### 19. 为summary准备路径

若/tmp/retain_dir 存在就删掉重新保存，若要保存中间的训练过程而不只是只保存最终结果就创建folder /tmp/intermediate_out_graph

In [ ]:
def prepare_file_system():
  # Setup the directory we'll write summaries to for TensorBoard
  if tf.gfile.Exists(FLAGS.summaries_dir):
    tf.gfile.DeleteRecursively(FLAGS.summaries_dir)
  tf.gfile.MakeDirs(FLAGS.summaries_dir)
  if FLAGS.intermediate_store_frequency > 0:
    ensure_dir_exists(FLAGS.intermediate_output_graphs_dir)
  return

### 20. 输入模型名称，返回模型的信息

定义模型的信息，因为可以加载不同的模型，也就是通过architecture指定，"inception_v3" 或者 'mobilenet_something'; 以inception_v3模型为例的返回是：
 return {
      'data_url': 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz',
      'bottleneck_tensor_name': 'pool_3/_reshape:0',
      'bottleneck_tensor_size': 2048,
      'input_width': 299,
      'input_height': 299,
      'input_depth': 3,
      'resized_input_tensor_name': 'Mul:0',
      'model_file_name': 'classify_image_graph_def.pb',
      'input_mean': 128,
      'input_std': 128,
  }
  具体每一项都是干嘛的，还不清楚。

In [ ]:
def create_model_info(architecture):
  """Given the name of a model architecture, returns information about it.
  There are different base image recognition pretrained models that can be
  retrained using transfer learning, and this function translates from the name
  of a model to the attributes that are needed to download and train with it.
  Args:
    architecture: Name of a model architecture.
  Returns:
    Dictionary of information about the model, or None if the name isn't
    recognized
  Raises:
    ValueError: If architecture name is unknown.
  """
  architecture = architecture.lower()
  if architecture == 'inception_v3':
    # pylint: disable=line-too-long
    data_url = 'http://download.tensorflow.org/models/image/imagenet/inception-2015-12-05.tgz'
    # pylint: enable=line-too-long
    bottleneck_tensor_name = 'pool_3/_reshape:0'
    bottleneck_tensor_size = 2048
    input_width = 299
    input_height = 299
    input_depth = 3
    resized_input_tensor_name = 'Mul:0'
    model_file_name = 'classify_image_graph_def.pb'
    input_mean = 128
    input_std = 128
  elif architecture.startswith('mobilenet_'):
    parts = architecture.split('_')
    if len(parts) != 3 and len(parts) != 4:
      tf.logging.error("Couldn't understand architecture name '%s'",
                       architecture)
      return None
    version_string = parts[1]
    if (version_string != '1.0' and version_string != '0.75' and
        version_string != '0.50' and version_string != '0.25'):
      tf.logging.error(
          """"The Mobilenet version should be '1.0', '0.75', '0.50', or '0.25',
  but found '%s' for architecture '%s'""",
          version_string, architecture)
      return None
    size_string = parts[2]
    if (size_string != '224' and size_string != '192' and
        size_string != '160' and size_string != '128'):
      tf.logging.error(
          """The Mobilenet input size should be '224', '192', '160', or '128',
 but found '%s' for architecture '%s'""",
          size_string, architecture)
      return None
    if len(parts) == 3:
      is_quantized = False
    else:
      if parts[3] != 'quantized':
        tf.logging.error(
            "Couldn't understand architecture suffix '%s' for '%s'", parts[3],
            architecture)
        return None
      is_quantized = True
    data_url = 'http://download.tensorflow.org/models/mobilenet_v1_'
    data_url += version_string + '_' + size_string + '_frozen.tgz'
    bottleneck_tensor_name = 'MobilenetV1/Predictions/Reshape:0'
    bottleneck_tensor_size = 1001
    input_width = int(size_string)
    input_height = int(size_string)
    input_depth = 3
    resized_input_tensor_name = 'input:0'
    if is_quantized:
      model_base_name = 'quantized_graph.pb'
    else:
      model_base_name = 'frozen_graph.pb'
    model_dir_name = 'mobilenet_v1_' + version_string + '_' + size_string
    model_file_name = os.path.join(model_dir_name, model_base_name)
    input_mean = 127.5
    input_std = 127.5
  else:
    tf.logging.error("Couldn't understand architecture name '%s'", architecture)
    raise ValueError('Unknown architecture', architecture)

  return {
      'data_url': data_url,
      'bottleneck_tensor_name': bottleneck_tensor_name,
      'bottleneck_tensor_size': bottleneck_tensor_size,
      'input_width': input_width,
      'input_height': input_height,
      'input_depth': input_depth,
      'resized_input_tensor_name': resized_input_tensor_name,
      'model_file_name': model_file_name,
      'input_mean': input_mean,
      'input_std': input_std,
  }

### 21. 根据模型信息预处理图像

(1) 将图像resize成[input_width, input_height, input_depth] -> [299, 299, 3], 这里就用到了上面的模型信息；
(2) 将所有像素值减去input_mean: offset_image = resized_image - 128;
(3) 归一化图像：mul_image = offset_image / 128.0；
返回传入图像的placeholder: jpeg_data, 处理后的图像mul_image；
函数16中的add_input_distortions的最后部分做了相同的事情，不同的是add_input_distortions还做了distortation操作；
这个具体实现的时候可以作为预处理放在训练之外，比如用opencv实现pre_process

In [ ]:
def add_jpeg_decoding(input_width, input_height, input_depth, input_mean,
                      input_std):
  """Adds operations that perform JPEG decoding and resizing to the graph..
  Args:
    input_width: Desired width of the image fed into the recognizer graph.
    input_height: Desired width of the image fed into the recognizer graph.
    input_depth: Desired channels of the image fed into the recognizer graph.
    input_mean: Pixel value that should be zero in the image for the graph.
    input_std: How much to divide the pixel values by before recognition.
  Returns:
    Tensors for the node to feed JPEG data into, and the output of the
      preprocessing steps.
  """
  jpeg_data = tf.placeholder(tf.string, name='DecodeJPGInput')#placeholder用来传入gfile string
  decoded_image = tf.image.decode_jpeg(jpeg_data, channels=input_depth)
  decoded_image_as_float = tf.cast(decoded_image, dtype=tf.float32)
  decoded_image_4d = tf.expand_dims(decoded_image_as_float, 0)
  resize_shape = tf.stack([input_height, input_width])
  resize_shape_as_int = tf.cast(resize_shape, dtype=tf.int32)
  resized_image = tf.image.resize_bilinear(decoded_image_4d,
                                           resize_shape_as_int)
  offset_image = tf.subtract(resized_image, input_mean)
  mul_image = tf.multiply(offset_image, 1.0 / input_std)
  return jpeg_data, mul_image

### 22. main函数

In [ ]:
def main(_):
  # Needed to make sure the logging output is visible.
  # See https://github.com/tensorflow/tensorflow/issues/3047
  tf.logging.set_verbosity(tf.logging.INFO)# 确保tf.logging能输出

  # Prepare necessary directories  that can be used during training
  prepare_file_system() # 创建summary的路径

  # Gather information about the model architecture we'll be using.
  model_info = create_model_info(FLAGS.architecture)#输入模型名称，返回模型信息的字典
  if not model_info:
    tf.logging.error('Did not recognize architecture flag') #信息获取失败，返回
    return -1

  # Set up the pre-trained graph.
  maybe_download_and_extract(model_info['data_url']) # 若模型没有在默认路径中，则下载并解压
  # 根据模型信息和下载好的模型加载训练好的计算图，返回计算图，用来计算bottleneck和输入图像的两个tensor
  graph, bottleneck_tensor, resized_image_tensor = (
      create_model_graph(model_info))

  # Look at the folder structure, and create lists of all the images.
  image_lists = create_image_lists(FLAGS.image_dir, FLAGS.testing_percentage,
                                   FLAGS.validation_percentage) #创建所有图像的字典，并按照testing和validation的百分比随机分
  class_count = len(image_lists.keys()) # 分类的个数，label_name的个数
  if class_count == 0:
    tf.logging.error('No valid folders of images found at ' + FLAGS.image_dir)#分类个数为0说明没有正确组织结构
    return -1
  if class_count == 1:
    tf.logging.error('Only one valid folder of images found at ' +
                     FLAGS.image_dir +
                     ' - multiple classes are needed for classification.')#分类个数为1，那么也没必要分类
    return -1

  # See if the command-line flags mean we're applying any distortions.
    # 判断是否需要distort, 返回一个bool值
  do_distort_images = should_distort_images(
      FLAGS.flip_left_right, FLAGS.random_crop, FLAGS.random_scale,
      FLAGS.random_brightness)
  # 所以这里的graph就是上述加载好的与训练模型
  with tf.Session(graph=graph) as sess:
    # Set up the image decoding sub-graph.
    # 新添加的两个tensor，一个用来传入jpeg string，另外一个是按照模型信息进行resize和归一化之后的图像tensor
    # image_string_data ->feed-> jpeg_data_tensor -> decoded_image_tensor -> model -> bottleneck
    jpeg_data_tensor, decoded_image_tensor = add_jpeg_decoding(
        model_info['input_width'], model_info['input_height'],
        model_info['input_depth'], model_info['input_mean'],
        model_info['input_std'])

    if do_distort_images:
      # We will be applying distortions, so setup the operations we'll need.
      # 新添加两个tensor, 一个用来传入待变化的图像，一个保存图像变化后的结果，这个函数在图像变换后做了和add_jpeg_decoding一样的事情
      # 将图像resize到模型需要的尺寸，返回一个接收图像string的distorted_jpeg_data_tensor，和变换和resize处理后的distorted_image_tensor
      # 所以当需要随机变换图像时：
      # image_string_data ->feed-> distorted_jpeg_data_tensor -> distorted_image_tensor -> 
      # ->feed-> resized_image_tensor -> bottleneck_tensor -> bottleneck
      (distorted_jpeg_data_tensor,
       distorted_image_tensor) = add_input_distortions(
           FLAGS.flip_left_right, FLAGS.random_crop, FLAGS.random_scale,
           FLAGS.random_brightness, model_info['input_width'],
           model_info['input_height'], model_info['input_depth'],
           model_info['input_mean'], model_info['input_std'])
    else:
      # We'll make sure we've calculated the 'bottleneck' image summaries and
      # cached them on disk.
      # 若不需要变化，对所有的上述按照模型信息进行resize和归一化后的图像进行bottleneck生成
      # image_string_data ->feed-> jpeg_data_tensor -> decoded_image_tensor 
      # ->feed-> resized_image_tensor -> model -> bottleneck
      cache_bottlenecks(sess, image_lists, FLAGS.image_dir,
                        FLAGS.bottleneck_dir, jpeg_data_tensor,
                        decoded_image_tensor, resized_image_tensor,
                        bottleneck_tensor, FLAGS.architecture)

    # Add the new layer that we'll be training.
    # 根据bottleneck_tensor返回新加层的运算op和placeholder
    # bottleneck_input 是输入placeholder[None, bottleneck_tensor_size(2048 or 1001)]
    # ground_truth_input 是输入placeholder[None, len(image_lists.keys())=5]
    # cross_entropy 是损失函数，需要上述两个placeholder输入的数据；
    # train_step 是cross_entropy的优化op；
    # final_tensor 是softmax(logits), 同样需要上述两个placeholder feed
    (train_step, cross_entropy, bottleneck_input, ground_truth_input,
     final_tensor) = add_final_training_ops(
         len(image_lists.keys()), FLAGS.final_tensor_name, bottleneck_tensor,
         model_info['bottleneck_tensor_size'])

    # Create the operations we need to evaluate the accuracy of our new layer.
    # 输入预测输出final_tensor和ground_truth，比较两者计算正确率-> evaluation_step
    # prediction是简单的argmax(final_tensor)
    evaluation_step, prediction = add_evaluation_step(
        final_tensor, ground_truth_input)

    # Merge all the summaries and write them out to the summaries_dir
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter(FLAGS.summaries_dir + '/train',
                                         sess.graph)

    validation_writer = tf.summary.FileWriter(
        FLAGS.summaries_dir + '/validation')

    # Set up all our weights to their initial default values.
    init = tf.global_variables_initializer()
    sess.run(init)

    # Run the training for as many cycles as requested on the command line.
    # 训练过程开始，迭代how_many_training_steps＝4000次
    for i in range(FLAGS.how_many_training_steps):
      # Get a batch of input bottleneck values, either calculated fresh every
      # time with distortions applied, or from the cache stored on disk.
      if do_distort_images:
        # 如果需要变化图像，就需要先变化再重新生成bottleneck组成train_bottlenecks 和 train_ground_truth
        (train_bottlenecks,
         train_ground_truth) = get_random_distorted_bottlenecks(
             sess, image_lists, FLAGS.train_batch_size, 'training',
             FLAGS.image_dir, distorted_jpeg_data_tensor,
             distorted_image_tensor, resized_image_tensor, bottleneck_tensor)
      else:
        # 如果不需要变换图像，则读取预先生成的bottlenecks
        (train_bottlenecks,
         train_ground_truth, _) = get_random_cached_bottlenecks(
             sess, image_lists, FLAGS.train_batch_size, 'training',
             FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
             decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
             FLAGS.architecture)
      # Feed the bottlenecks and ground truth into the graph, and run a training
      # step. Capture training summaries for TensorBoard with the `merged` op.
      train_summary, _ = sess.run(
          [merged, train_step],
          feed_dict={bottleneck_input: train_bottlenecks,
                     ground_truth_input: train_ground_truth})
      train_writer.add_summary(train_summary, i)

      # Every so often, print out how well the graph is training.
      # 每隔10步输出训练准确率，训练损失，validation准确率
      is_last_step = (i + 1 == FLAGS.how_many_training_steps)
      if (i % FLAGS.eval_step_interval) == 0 or is_last_step:
        train_accuracy, cross_entropy_value = sess.run(
            [evaluation_step, cross_entropy],
            feed_dict={bottleneck_input: train_bottlenecks,
                       ground_truth_input: train_ground_truth})
        tf.logging.info('%s: Step %d: Train accuracy = %.1f%%' %
                        (datetime.now(), i, train_accuracy * 100))
        tf.logging.info('%s: Step %d: Cross entropy = %f' %
                        (datetime.now(), i, cross_entropy_value))
        validation_bottlenecks, validation_ground_truth, _ = (
            get_random_cached_bottlenecks(
                sess, image_lists, FLAGS.validation_batch_size, 'validation',
                FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
                decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
                FLAGS.architecture))
        # Run a validation step and capture training summaries for TensorBoard
        # with the `merged` op.
        validation_summary, validation_accuracy = sess.run(
            [merged, evaluation_step],
            feed_dict={bottleneck_input: validation_bottlenecks,
                       ground_truth_input: validation_ground_truth})
        validation_writer.add_summary(validation_summary, i)
        tf.logging.info('%s: Step %d: Validation accuracy = %.1f%% (N=%d)' %
                        (datetime.now(), i, validation_accuracy * 100,
                         len(validation_bottlenecks)))

      # Store intermediate results
      intermediate_frequency = FLAGS.intermediate_store_frequency
    
      # 如果想每隔intermediate_frequency步保存一下模型，指定intermediate_frequency一个大于0的值即可；
      if (intermediate_frequency > 0 and (i % intermediate_frequency == 0)
          and i > 0):
        intermediate_file_name = (FLAGS.intermediate_output_graphs_dir +
                                  'intermediate_' + str(i) + '.pb')
        tf.logging.info('Save intermediate result to : ' +
                        intermediate_file_name)
        save_graph_to_file(sess, graph, intermediate_file_name)

    # We've completed all our training, so run a final test evaluation on
    # some new images we haven't used before.
    # 全部训练完毕，获取全部的testing bottlenecks 测试准确率。
    test_bottlenecks, test_ground_truth, test_filenames = (
        get_random_cached_bottlenecks(
            sess, image_lists, FLAGS.test_batch_size, 'testing',
            FLAGS.bottleneck_dir, FLAGS.image_dir, jpeg_data_tensor,
            decoded_image_tensor, resized_image_tensor, bottleneck_tensor,
            FLAGS.architecture))
    test_accuracy, predictions = sess.run(
        [evaluation_step, prediction],
        feed_dict={bottleneck_input: test_bottlenecks,
                   ground_truth_input: test_ground_truth})
    tf.logging.info('Final test accuracy = %.1f%% (N=%d)' %
                    (test_accuracy * 100, len(test_bottlenecks)))
    # 定义是否输出错误分类的图像名称
    if FLAGS.print_misclassified_test_images:
      tf.logging.info('=== MISCLASSIFIED TEST IMAGES ===')
      for i, test_filename in enumerate(test_filenames):
        if predictions[i] != test_ground_truth[i].argmax():
          tf.logging.info('%70s  %s' %
                          (test_filename,
                           list(image_lists.keys())[predictions[i]]))

    # Write out the trained graph and labels with the weights stored as
    # constants.
    save_graph_to_file(sess, graph, FLAGS.output_graph)
    with gfile.FastGFile(FLAGS.output_labels, 'w') as f:
      f.write('\n'.join(image_lists.keys()) + '\n')

